<H3>Import and format data</H3>
<p>First we will import the weather data from the repository. The weather data contains the daily maximum and minimum temperatures and the rainfall between June 2018 and July 2019.</p>

In [205]:
import pandas as pd

weather = pd.read_csv('seasons.csv', index_col='Date')
weather.head()

,Max,Min,Rain
Date,,,
1/06/2018,17.7,4.0,0.0
2/06/2018,19.4,5.4,0.0
3/06/2018,17.2,4.2,0.0
4/06/2018,15.2,7.3,0.0
5/06/2018,15.6,3.9,0.0


<p>A quick check shows that there are no missing values, but does reveal the date column is of the object file type. We want to convert the file type to the datetime64 format so that rows can be selected between certain dates, i.e between seasons.</p>

In [206]:
weather.info()
weather.index.format

<class 'pandas.core.frame.DataFrame'>
Index: 365 entries, 1/06/2018 to 31/05/2019
Data columns (total 3 columns):
Max     365 non-null float64
Min     365 non-null float64
Rain    365 non-null float64
dtypes: float64(3)
memory usage: 11.4+ KB


<bound method Index.format of Index(['1/06/2018', '2/06/2018', '3/06/2018', '4/06/2018', '5/06/2018',
       '6/06/2018', '7/06/2018', '8/06/2018', '9/06/2018', '10/06/2018',
       ...
       '22/05/2019', '23/05/2019', '24/05/2019', '25/05/2019', '26/05/2019',
       '27/05/2019', '28/05/2019', '29/05/2019', '30/05/2019', '31/05/2019'],
      dtype='object', name='Date', length=365)>

<p>The Pandas method to_datetime can be used to convert the dates into the datetime64 format. Observe that the day appears at the front of the date, dd/mm/yyy - to ensure Pandas parses the dates correctly, the dayfirst attribute must be set to True. After this operation the date format can be seen to be datetime64.</p>

In [207]:
weather.index = pd.to_datetime(weather.index, dayfirst=True)
print(weather.index.format)
weather.head()

<bound method Index.format of DatetimeIndex(['2018-06-01', '2018-06-02', '2018-06-03', '2018-06-04',
               '2018-06-05', '2018-06-06', '2018-06-07', '2018-06-08',
               '2018-06-09', '2018-06-10',
               ...
               '2019-05-22', '2019-05-23', '2019-05-24', '2019-05-25',
               '2019-05-26', '2019-05-27', '2019-05-28', '2019-05-29',
               '2019-05-30', '2019-05-31'],
              dtype='datetime64[ns]', name='Date', length=365, freq=None)>


,Max,Min,Rain
Date,,,
2018-06-01,17.7,4.0,0.0
2018-06-02,19.4,5.4,0.0
2018-06-03,17.2,4.2,0.0
2018-06-04,15.2,7.3,0.0
2018-06-05,15.6,3.9,0.0


<h3>Prepare data for plotting</h3>
<p>The final result will plot a filled line bounded by the maximum and minimum temperatures of each day and divided by colour into the four seasons. To achieve this look, each season is plotted as individual sections:</p>
<list>
    <li>Winter: 1 June to 31 August</li>
    <li>Sprint: 1 September to 30 November</li>
    <li>Summer: 1 December to 28 February</li>
    <li>Autumn: 1 March to 31 May</li>
</list>
<p>A day overlap is introduced to keep consistancy between each section of the plot.</p>
<p>To get the filled line effect for each season, the plotted line must first make its way from the left to the right of the season, then move its way back from the right to the left. This is done so that the fill attribute 'toself' fills between the max and min line.</p>
<p> The [::-1] range selector reverses the order of the list. This is used to place the reversed order list of minimum values on the end of the maximum values, so that when they are plotted they end up as once continuous line. This must also be done for the second half of the x values so that the max and mins position correctly. The result is shown in the output of the below cell.</p>
   

In [209]:
winter_max = weather.loc['2018-06-01':'2018-09-01'].Max.tolist()
winter_min = weather.loc['2018-06-01':'2018-09-01'].Min.tolist()
winter_x = weather.loc['2018-06-01':'2018-09-01'].index.tolist()

winter_temp = winter_max + winter_min[::-1]
winter_x = winter_x + winter_x[::-1]

fig = go.Figure()

fig.add_trace(go.Scatter(x=winter_x, y=winter_temp,
    fill='none',
    mode='lines',
    line_color='blue',
    name='Winter'
    ))

py.offline.iplot(fig)


<p>The same process is carried out for the remainder of the seasons. The rain data is not broken down into seasons and is plotted as one continuous line.</p>

In [ ]:
spring_max = weather.loc['2018-09-01':'2018-12-01'].Max.tolist()
spring_min = weather.loc['2018-09-01':'2018-12-01'].Min.tolist()
spring_x = weather.loc['2018-09-01':'2018-12-01'].index.tolist()

spring_temp = spring_max + spring_min[::-1]
spring_x = spring_x + spring_x[::-1]


summer_max = weather.loc['2018-12-01':'2019-03-01'].Max.tolist()
summer_min = weather.loc['2018-12-01':'2019-03-01'].Min.tolist()
summer_x = weather.loc['2018-12-01':'2019-03-01'].index.tolist()

summer_temp = summer_max + summer_min[::-1]
summer_x = summer_x + summer_x[::-1]


autumn_max = weather.loc['2019-03-01':'2019-06-01'].Max.tolist()
autumn_min = weather.loc['2019-03-01':'2019-06-01'].Min.tolist()
autumn_x = weather.loc['2019-03-01':'2019-06-01'].index.tolist()

autumn_temp = autumn_max + autumn_min[::-1]
autumn_x = autumn_x + autumn_x[::-1]


rain = weather.Rain.tolist()
rain_x = weather.index.tolist()

<h3>Generate final plot</h3>
<p>The plot is generated as subplots that share the same y axis (date). The figure is set up as a subplot of 4 rows and 1 column, with the specs attribute set to make the temperature plot bigger by taking up 3 row positions. The fill attribute of each trace is set to 'toself' this time and coloured for the season.</p>

In [196]:
import plotly as py
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode
from plotly.subplots import make_subplots

init_notebook_mode(connected=True)

fig = make_subplots(
    rows=4, cols=1, shared_xaxes=True, vertical_spacing=0.01,
    specs=[[{"rowspan": 3}],
           [None],
           [None],
           [{}]]      
)

fig.add_trace(go.Scatter(x=winter_x, y=winter_temp,
    fill='toself',
    mode='lines',
    line_color='blue',
    name='Winter'),
    row=1,col=1)

fig.add_trace(go.Scatter(x=spring_x, y=spring_temp,
    fill='toself',
    mode='lines',
    line_color='green',
    name='Spring'),
    row=1,col=1)

fig.add_trace(go.Scatter(x=summer_x, y=summer_temp,
    fill='toself',
    mode='lines',
    line_color='red',
    name='Summer'),
    row=1,col=1)

fig.add_trace(go.Scatter(x=autumn_x, y=autumn_temp,
    fill='toself',
    mode='lines',
    line_color='orange',
    name='Autumn'),
    row=1,col=1)


fig.add_trace(go.Scatter(
    x=rain_x,
    y=rain,
    fill='none',
    mode='lines',
    line_color='blue',
    name='Rain'),
    row=4,col=1)

fig.update_yaxes(title_text="Temperature (\u00B0C)", title_font=dict(size=11), row=1, col=1)
fig.update_yaxes(title_text="Rainfall mm", title_font=dict(size=11), row=4, col=1)

fig.update_layout(
    title=go.layout.Title(
        text="Adelaide, Australia - Temperature and Rainfall"
    ))


py.offline.iplot(fig)